In [43]:
%pip install torch
%pip install numpy
%pip install pandas
%pip install openpyxl
%pip install onnx
%pip install onnxscript

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Read raw data

In [44]:
import json
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [45]:
rawData = pd.read_excel('data.xlsx', header=None, names=['Id', 'BotInput', 'BotCommand'])


In [46]:
print("Number of botinputs",len(rawData))
rawData.head(5)

Number of botinputs 7524


,Id,BotInput,BotCommand
0,"{""$oid"":""66ab864800d3c31312357141""}","{""Player"":{""Position"":{""X"":-7.538232803,""Y"":3....","{""Move"":{""X"":0.0,""Y"":0.0,""SqrMagnitude"":0.0},""..."
1,"{""$oid"":""66ab864800d3c31312357142""}","{""Player"":{""Position"":{""X"":-7.538232803,""Y"":3....","{""Move"":{""X"":0.0,""Y"":0.0,""SqrMagnitude"":0.0},""..."
2,"{""$oid"":""66ab864800d3c31312357143""}","{""Player"":{""Position"":{""X"":-7.538232803,""Y"":3....","{""Move"":{""X"":0.0,""Y"":0.0,""SqrMagnitude"":0.0},""..."
3,"{""$oid"":""66ab864800d3c31312357144""}","{""Player"":{""Position"":{""X"":-7.538232803,""Y"":3....","{""Move"":{""X"":0.0,""Y"":0.0,""SqrMagnitude"":0.0},""..."
4,"{""$oid"":""66ab864800d3c31312357145""}","{""Player"":{""Position"":{""X"":-7.538232803,""Y"":3....","{""Move"":{""X"":0.0,""Y"":0.0,""SqrMagnitude"":0.0},""..."


Data visualization 

Clean data

In [47]:


def preprocess_botcommand(data):
 

  data_list = []
  for item in data:
    # Extract numerical features
    parsed_data = json.loads(item)

    move_x = parsed_data['Move']['X']
    move_y = parsed_data['Move']['Y']
    rotate = parsed_data['Rotate']
    shoot =  parsed_data['Shoot']
    # Create feature vector
    feature_vector = [move_x, move_y, rotate,shoot] 
    data_list.append(feature_vector)

  df = pd.DataFrame(data_list, columns=['Move_X', 'Move_Y', 'Rotate','Shoot']) 
  return df

def get_xy_position(json) : 
  return json['Position']['X'] , json['Position']['Y']

def preprocess_botinput(input) :

  #Player Features
  parsed_data = json.loads(input)
  player_x , player_y = get_xy_position( parsed_data['Player'])

  player_velocity = parsed_data['Player']['Velocity']['SqrMagnitude']
  player_rotation = parsed_data['Player']['Rotation']
  player_health = parsed_data['Player']['Health']
  player_score = parsed_data['Player']['Score']

  # Bullet
  bullet_x = -20
  bullet_y = -20

  if parsed_data["Bullets"] != [] :
    bullet_x , bullet_y = get_xy_position( (parsed_data['Bullets'][0]))

  #OtherPlayer
  other_player_x = -20
  other_player_y = -20
  other_player_velocity = 0
  other_player_rotation = 0
  if parsed_data["OtherPlayers"] != [] :
    other_player_x , other_player_y = get_xy_position((parsed_data['OtherPlayers'][0]))
    other_player_velocity = (parsed_data['OtherPlayers'][0])['Velocity']['SqrMagnitude']
    player_rotation = (parsed_data['OtherPlayers'][0])['Rotation']

  #Obstacles
  obstacle_x = -20
  obstacle_y = -20

  if parsed_data["Obstacles"] != [] :
    obstacle_x , obstacle_y = get_xy_position( (parsed_data['Obstacles'][0]))

  feature_vector = [player_x, player_y, player_velocity,player_rotation,player_health,player_score, 
                    bullet_x,bullet_y,other_player_x,other_player_y,other_player_velocity,other_player_rotation,obstacle_x,obstacle_y ]


  return torch.tensor(feature_vector, dtype=torch.float32)

In [48]:
labels = preprocess_botcommand(rawData["BotCommand"])
#features = preprocess_botinput(rawData["BotInput"][780])

Model definition

In [49]:
class BotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size_move, output_size_rotate):
        super(BotModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size,hidden_size*2)
        
        self.fc_move = nn.Linear(hidden_size*2, output_size_move)
        self.fc_rotate = nn.Linear(hidden_size*2, output_size_rotate)
        self.fc_shoot = nn.Linear(hidden_size*2, 1)  # For classification

    def movement_nextwork(self,x):
        
        move =  torch.relu(self.fc_move(x))
        return move
    
    def rotation_nextwork(self,x):
        rotate = torch.tanh(self.fc_rotate(x))
        return rotate
    
    def shoot_nextwork(self,x):
        shoot = torch.sigmoid(self.fc_shoot(x))
        return shoot 
    
    def forward(self, x):   
         # For classification
        x = torch.relu(self.fc1(x))
        x =  self.fc2(x)
        return self.movement_nextwork(x) , self.rotation_nextwork(x) , self.shoot_nextwork(x)
    

Training 

In [50]:
input_size = preprocess_botinput(rawData["BotInput"][0]).shape[0]  # Replace with actual input size
hidden_size = 64
output_size_move = 2
output_size_rotate = 1
num_epochs = 10
learning_rate = 0.001

In [51]:
def to_tensor(data , dtype = torch.float32):
    return torch.tensor(data,dtype=dtype)
    
def train(model, X,x_move,y_move, rotate, shoot, num_epochs, learning_rate):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()
    loss_fn_shoot = nn.BCELoss()

    for epoch in range(num_epochs):
        for i in range(len(X)):
            optimizer.zero_grad()
            move_pred , rotate_pred , shoot_pred = model(X[i]) 
            loss_move= loss_fn(move_pred, torch.tensor([x_move[i] , y_move[i]],dtype=torch.float32))
            loss_rotate = loss_fn(rotate_pred ,rotate[i])
            loss_shoot = loss_fn_shoot( shoot_pred[0] , shoot[i] )  # Convert to float for BCELoss
            loss =   loss_shoot + loss_rotate +loss_move
            loss.backward()
            optimizer.step()
            print("Loss at Epoch ",epoch,"is", loss.item())

In [52]:
model_data = rawData["BotInput"].apply(preprocess_botinput)

x_move = torch.tensor(labels['Move_X'], dtype=torch.float32)
y_move = torch.tensor(labels['Move_Y'], dtype=torch.float32)

rotate = torch.tensor(labels['Rotate'], dtype=torch.float32)
shoot = torch.tensor(labels['Shoot'], dtype=torch.float32)
model = BotModel(input_size, hidden_size, output_size_move, output_size_rotate)

train(model, model_data.values, x_move,y_move, rotate, shoot, num_epochs, learning_rate)


d:\apps\miniconda3\Lib\site-packages\torch\nn\modules\loss.py:538: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Loss at Epoch  0 is 1.1871931552886963
Loss at Epoch  0 is 0.33215972781181335
Loss at Epoch  0 is 0.30689164996147156
Loss at Epoch  0 is 0.1713661551475525
Loss at Epoch  0 is 0.0953533798456192
Loss at Epoch  0 is 0.10195950418710709
Loss at Epoch  0 is 0.05537295341491699
Loss at Epoch  0 is 0.0276582483202219
Loss at Epoch  0 is 0.04599874094128609
Loss at Epoch  0 is 0.03393096476793289
Loss at Epoch  0 is 0.010078204795718193
Loss at Epoch  0 is 0.019727852195501328
Loss at Epoch  0 is 0.02586168795824051
Loss at Epoch  0 is 0.009249206632375717
Loss at Epoch  0 is 0.00631374679505825
Loss at Epoch  0 is 0.017180558294057846
Loss at Epoch  0 is 0.011682499200105667
Loss at Epoch  0 is 0.0023471831809729338
Loss at Epoch  0 is 0.008517373353242874
Loss at Epoch  0 is 0.011395112611353397
Loss at Epoch  0 is 0.0033197826705873013
Loss at Epoch  0 is 0.003094702959060669
Loss at Epoch  0 is 0.008518030866980553
Loss at Epoch  0 is 0.0048487503081560135
Loss at Epoch  0 is 0.0011343

In [53]:
onnx_program = torch.onnx.dynamo_export(model, model_data.values[0])
 
onnx_program.save("bot.onnx")

Applied 5 of general pattern rewrite rules.


d:\apps\miniconda3\Lib\site-packages\torch\onnx\_internal\exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


Evaluations